In [1]:
import pyspark

In [2]:
from pyspark.sql import SparkSession

In [3]:
#loading the required sql functions for the further processing
from pyspark.sql import functions
from pyspark.sql.functions import col, trim, lower, regexp_replace

In [4]:
spark = SparkSession.builder \
    .appName("BookRecommendEDA") \
    .getOrCreate()

25/07/21 10:23:00 WARN Utils: Your hostname, vaibhavi-HP-Laptop-15-fd0xxx resolves to a loopback address: 127.0.1.1; using 172.18.4.62 instead (on interface wlo1)
25/07/21 10:23:00 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/07/21 10:23:01 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/07/21 10:23:02 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [5]:
df_reviews = spark.read.csv("/home/vaibhavi/spark-ml-venv/ml_project/data/Books_rating.csv", header=True, inferSchema=True)
df_reviews.printSchema()
df_reviews.show(5)

root
 |-- Id: string (nullable = true)
 |-- Title: string (nullable = true)
 |-- Price: string (nullable = true)
 |-- User_id: string (nullable = true)
 |-- profileName: string (nullable = true)
 |-- review/helpfulness: string (nullable = true)
 |-- review/score: string (nullable = true)
 |-- review/time: string (nullable = true)
 |-- review/summary: string (nullable = true)
 |-- review/text: string (nullable = true)

+----------+--------------------+-----+--------------+--------------------+------------------+------------+-----------+--------------------+--------------------+
|        Id|               Title|Price|       User_id|         profileName|review/helpfulness|review/score|review/time|      review/summary|         review/text|
+----------+--------------------+-----+--------------+--------------------+------------------+------------+-----------+--------------------+--------------------+
|1882931173|Its Only Art If I...| NULL| AVCGYZL8FQQTD|"Jim of Oz ""jim-...|               7/

In [6]:
df_books = spark.read.csv("/home/vaibhavi/spark-ml-venv/ml_project/data/books_data.csv", header=True, inferSchema=True)
df_books.printSchema()
df_books.show(5)

root
 |-- Title: string (nullable = true)
 |-- description: string (nullable = true)
 |-- authors: string (nullable = true)
 |-- image: string (nullable = true)
 |-- previewLink: string (nullable = true)
 |-- publisher: string (nullable = true)
 |-- publishedDate: string (nullable = true)
 |-- infoLink: string (nullable = true)
 |-- categories: string (nullable = true)
 |-- ratingsCount: string (nullable = true)

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------+--------------------+--------------------+------------+
|               Title|         description|             authors|               image|         previewLink|           publisher| publishedDate|            infoLink|          categories|ratingsCount|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------+--------------------+--------------------+---

actual Schema : Title,description,authors,image,previewLink,publisher,publishedDate,infoLink,categories,ratingsCount

#required columns from both the tables:
- title, userId, review_score, tb2 - title, authors ,publisher, categories,description, ratingsCount

In [7]:
#selecting required columns from books dataframe
df_required1 = df_books.select("Title","description","authors","publisher", "categories","ratingsCount")

In [9]:
review_counts = df_reviews.groupBy("Title").count().withColumnRenamed("count", "review_count")

review_counts.show()



+--------------------+------------+
|               Title|review_count|
+--------------------+------------+
|Isaac Asimov: Mas...|           2|
|     White Rock Ways|           1|
|The Face of the T...|           1|
|Your Signature Li...|           4|
|     Iridescent Soul|           8|
|L'Alchimiste (Cof...|          13|
|  The Book of Garlic|           1|
|A Jesse Stuart Ha...|           1|
|Raymond Chandler:...|          15|
|      Badenheim 1939|          26|
|        Pagan Babies|          16|
|The Self and its ...|           3|
|The Educated Chil...|          37|
|Future Perfect - ...|           2|
|The cornet of hor...|          18|
|Basic Arabic Work...|           7|
|Organizational Th...|           5|
|Oz and Beyond: Th...|           1|
|Fundamentals of I...|           3|
|We Love Baseball!...|           2|
+--------------------+------------+
only showing top 20 rows



In [10]:
df_required1.printSchema()

root
 |-- Title: string (nullable = true)
 |-- description: string (nullable = true)
 |-- authors: string (nullable = true)
 |-- publisher: string (nullable = true)
 |-- categories: string (nullable = true)
 |-- ratingsCount: string (nullable = true)



In [11]:
df_required1 = df_required1.join(review_counts , on = ["Title"] ,how = 'left')

df_required1.printSchema()

root
 |-- Title: string (nullable = true)
 |-- description: string (nullable = true)
 |-- authors: string (nullable = true)
 |-- publisher: string (nullable = true)
 |-- categories: string (nullable = true)
 |-- ratingsCount: string (nullable = true)
 |-- review_count: long (nullable = true)



In [12]:
df_required1.count()

212404

In [20]:
df_reviews= df_reviews.drop("review/helpfulness","review/time","review/summary","review/text","Price","profileName")
df_reviews.printSchema()

root
 |-- Id: string (nullable = true)
 |-- Title: string (nullable = true)
 |-- User_id: string (nullable = true)
 |-- review/score: string (nullable = true)



In [51]:
df_als = df_reviews.select("Id","User_id","review/score")

In [34]:
df_als.count()

3000000

In [52]:
df_als = df_als.withColumnRenamed("review/score","rating")
df_als.printSchema()

root
 |-- Id: string (nullable = true)
 |-- User_id: string (nullable = true)
 |-- rating: string (nullable = true)



there are several duplicate values of same user giving different ratings to the same book, we will be averaging their ratings to have proper data, as als (collaborative filtering) requires distinct user_id, id and the rating triplet

In [53]:
#there are several duplicate values in the ratings column 
from pyspark.sql.functions import avg

df_als = df_als.groupBy("User_id", "Id").agg(avg("rating").alias("rating"))



In [54]:
from pyspark.sql.functions import col

df_als.filter(
    col("User_id").isNull() | col("Id").isNull() | col("rating").isNull()
).count()


96862

In [56]:
df_als = df_als.dropna(subset=["User_id", "Id", "rating"])



In [57]:
df_als.count()

2380346

In [68]:
#as we did with sentiment, storing the data in batches
num_batches = 10

# splitting into batches
batches = df_als.randomSplit([1.0] * num_batches, seed=42)


In [69]:
# Save each batch as a separate Parquet file
for i, batch in enumerate(batches):
    print(f"Saving batch {i+1} to Parquet...")
    batch.write \
        .mode("overwrite") \
        .parquet(f"output/als/als_parquet_batch_{i+1}")

Saving batch 1 to Parquet...


25/07/07 13:47:18 WARN MemoryManager: Total allocation exceeds 95.00% (982,201,127 bytes) of heap memory
Scaling row group sizes to 91.47% for 8 writers
25/07/07 13:47:18 WARN MemoryManager: Total allocation exceeds 95.00% (982,201,127 bytes) of heap memory
Scaling row group sizes to 81.31% for 9 writers
25/07/07 13:47:18 WARN MemoryManager: Total allocation exceeds 95.00% (982,201,127 bytes) of heap memory
Scaling row group sizes to 73.18% for 10 writers
25/07/07 13:47:18 WARN MemoryManager: Total allocation exceeds 95.00% (982,201,127 bytes) of heap memory
Scaling row group sizes to 66.53% for 11 writers
25/07/07 13:47:18 WARN MemoryManager: Total allocation exceeds 95.00% (982,201,127 bytes) of heap memory
Scaling row group sizes to 60.98% for 12 writers
25/07/07 13:47:18 WARN MemoryManager: Total allocation exceeds 95.00% (982,201,127 bytes) of heap memory
Scaling row group sizes to 66.53% for 11 writers
25/07/07 13:47:18 WARN MemoryManager: Total allocation exceeds 95.00% (982,201

Saving batch 2 to Parquet...


25/07/07 13:47:23 WARN MemoryManager: Total allocation exceeds 95.00% (982,201,127 bytes) of heap memory
Scaling row group sizes to 91.47% for 8 writers
25/07/07 13:47:23 WARN MemoryManager: Total allocation exceeds 95.00% (982,201,127 bytes) of heap memory
Scaling row group sizes to 81.31% for 9 writers
25/07/07 13:47:23 WARN MemoryManager: Total allocation exceeds 95.00% (982,201,127 bytes) of heap memory
Scaling row group sizes to 73.18% for 10 writers
25/07/07 13:47:23 WARN MemoryManager: Total allocation exceeds 95.00% (982,201,127 bytes) of heap memory
Scaling row group sizes to 66.53% for 11 writers
25/07/07 13:47:23 WARN MemoryManager: Total allocation exceeds 95.00% (982,201,127 bytes) of heap memory
Scaling row group sizes to 60.98% for 12 writers
25/07/07 13:47:23 WARN MemoryManager: Total allocation exceeds 95.00% (982,201,127 bytes) of heap memory
Scaling row group sizes to 66.53% for 11 writers
25/07/07 13:47:23 WARN MemoryManager: Total allocation exceeds 95.00% (982,201

Saving batch 3 to Parquet...


...

Saving batch 9 to Parquet...


25/07/07 13:48:05 WARN MemoryManager: Total allocation exceeds 95.00% (982,201,127 bytes) of heap memory
Scaling row group sizes to 91.47% for 8 writers
25/07/07 13:48:05 WARN MemoryManager: Total allocation exceeds 95.00% (982,201,127 bytes) of heap memory
Scaling row group sizes to 81.31% for 9 writers
25/07/07 13:48:05 WARN MemoryManager: Total allocation exceeds 95.00% (982,201,127 bytes) of heap memory
Scaling row group sizes to 73.18% for 10 writers
25/07/07 13:48:05 WARN MemoryManager: Total allocation exceeds 95.00% (982,201,127 bytes) of heap memory
Scaling row group sizes to 66.53% for 11 writers
25/07/07 13:48:05 WARN MemoryManager: Total allocation exceeds 95.00% (982,201,127 bytes) of heap memory
Scaling row group sizes to 60.98% for 12 writers
25/07/07 13:48:05 WARN MemoryManager: Total allocation exceeds 95.00% (982,201,127 bytes) of heap memory
Scaling row group sizes to 66.53% for 11 writers
25/07/07 13:48:05 WARN MemoryManager: Total allocation exceeds 95.00% (982,201

Saving batch 10 to Parquet...


25/07/07 13:48:12 WARN MemoryManager: Total allocation exceeds 95.00% (982,201,127 bytes) of heap memory
Scaling row group sizes to 91.47% for 8 writers
25/07/07 13:48:12 WARN MemoryManager: Total allocation exceeds 95.00% (982,201,127 bytes) of heap memory
Scaling row group sizes to 81.31% for 9 writers
25/07/07 13:48:12 WARN MemoryManager: Total allocation exceeds 95.00% (982,201,127 bytes) of heap memory
Scaling row group sizes to 73.18% for 10 writers
25/07/07 13:48:12 WARN MemoryManager: Total allocation exceeds 95.00% (982,201,127 bytes) of heap memory
Scaling row group sizes to 66.53% for 11 writers
25/07/07 13:48:12 WARN MemoryManager: Total allocation exceeds 95.00% (982,201,127 bytes) of heap memory
Scaling row group sizes to 60.98% for 12 writers
25/07/07 13:48:12 WARN MemoryManager: Total allocation exceeds 95.00% (982,201,127 bytes) of heap memory
Scaling row group sizes to 66.53% for 11 writers
25/07/07 13:48:12 WARN MemoryManager: Total allocation exceeds 95.00% (982,201

25/07/07 14:12:59 WARN TransportChannelHandler: Exception in connection from /192.168.1.9:34823
java.io.IOException: Connection timed out
	at sun.nio.ch.FileDispatcherImpl.read0(Native Method)
	at sun.nio.ch.SocketDispatcher.read(SocketDispatcher.java:39)
	at sun.nio.ch.IOUtil.readIntoNativeBuffer(IOUtil.java:223)
	at sun.nio.ch.IOUtil.read(IOUtil.java:192)
	at sun.nio.ch.SocketChannelImpl.read(SocketChannelImpl.java:379)
	at io.netty.buffer.PooledByteBuf.setBytes(PooledByteBuf.java:254)
	at io.netty.buffer.AbstractByteBuf.writeBytes(AbstractByteBuf.java:1132)
	at io.netty.channel.socket.nio.NioSocketChannel.doReadBytes(NioSocketChannel.java:357)
	at io.netty.channel.nio.AbstractNioByteChannel$NioByteUnsafe.read(AbstractNioByteChannel.java:151)
	at io.netty.channel.nio.NioEventLoop.processSelectedKey(NioEventLoop.java:788)
	at io.netty.channel.nio.NioEventLoop.processSelectedKeysOptimized(NioEventLoop.java:724)
	at io.netty.channel.nio.NioEventLoop.processSelectedKeys(NioEventLoop.java

##### now lets create EDA content-based required data  and check for the data in it. 

In [13]:
from pyspark.sql.functions import col, count, when, lit
from pyspark.sql.types import StringType

# Total number of rows in the DataFrame
total_rows = df_required1.count()


schema_info = [(f.name, f.dataType.simpleString()) for f in df_required1.schema.fields]

null_counts = df_required1.select([
    count(when(col(c).isNull(), c)).alias(c) for c, _ in schema_info
]).collect()[0].asDict()

# Create Spark DataFrame for summary
summary_data = [
    (col_name, dtype, null_counts[col_name], round(100 * null_counts[col_name] / total_rows, 2))
    for col_name, dtype in schema_info
]


summary_df = spark.createDataFrame(
    summary_data,
    ["column", "dtype", "null_count", "null_percent"]
)

summary_df.show(truncate=False)


+------------+------+----------+------------+
|column      |dtype |null_count|null_percent|
+------------+------+----------+------------+
|Title       |string|1         |0.0         |
|description |string|68357     |32.18       |
|authors     |string|31251     |14.71       |
|publisher   |string|73130     |34.43       |
|categories  |string|40524     |19.08       |
|ratingsCount|string|148552    |69.94       |
|review_count|bigint|1         |0.0         |
+------------+------+----------+------------+



In [14]:
# from the summary, ratings count is filled with lot of null,values, we will be dropping it, given that description is a text field 
# 32% empty would unncessary add overhead to the model for text processing ,so dropping description and publisher description as well

df_required1 = df_required1.drop("ratingsCount","description","publisher")

df_required1.printSchema()

root
 |-- Title: string (nullable = true)
 |-- authors: string (nullable = true)
 |-- categories: string (nullable = true)
 |-- review_count: long (nullable = true)



In [15]:
df_required1.count()

212404

In [18]:
df_required1.head(100)

[Row(Title='11,000 Years Lost', authors="['Peni R. Griffin']", categories="['Juvenile Fiction']", review_count=7),
 Row(Title='24 QUILTED GEMS: Sparkling Traditional and Original Projects', authors="['Gai Perry']", categories="['Crafts & Hobbies']", review_count=1),
 Row(Title='A book of remarkable criminals', authors="['Henry Brodribb Irving']", categories="['Crime']", review_count=12),
 Row(Title='ANPAO: AN AMERICAN INDIAN ODYSSEY', authors="['Jamake Highwater']", categories="['Juvenile Nonfiction']", review_count=16),
 Row(Title='Amanda Pig on Her Own', authors=None, categories=None, review_count=4),
 Row(Title='Applied Multivariate Statistics for the Social Sciences, Fifth Edition', authors="['James P. Stevens']", categories="['Education']", review_count=1),
 Row(Title='Are We Having Fun Yet?: The 16 Secrets of Happy Parenting', authors="['Kay Willis', 'Maryann Bucknum Brinley']", categories="['Family & Relationships']", review_count=9),
 Row(Title='BOOK OF COMMON PRAYER', authors=

In [19]:
from pyspark.sql.functions import col

# Only allow authors and categories that are either null or look like a list: ['Author1', 'Author2']
list_like_pattern = r"^\[.*\]$"

df_filtered = df_required1 \
    .filter(col("Title").isNotNull() & col("review_count").isNotNull()) \
    .filter(
        (col("authors").isNull() | col("authors").rlike(list_like_pattern)) &
        (col("categories").isNull() | col("categories").rlike(list_like_pattern))
    )


In [20]:
df_filtered.head()

Row(Title='"""Carefree"" (R.K.O.Classic Screenplays)"', authors="['Allan Scott', 'Ernest Pagano']", categories=None, review_count=1)

In [21]:
df_filtered.count()

188980

In [22]:
#as the data size is small, we will be only saving itno 2 batches

total_rows = df_filtered.count()
batch_size = total_rows // 2
batches = [
    df_filtered.limit(batch_size),
    df_filtered.subtract(df_filtered.limit(batch_size))
]


In [23]:
#storing in parquest agian

for i, batch in enumerate(batches):
    print(f"Saving content batch {i + 1}...")
    batch.write.mode("overwrite").parquet(f"output/content/content_batch_{i + 1}")


Saving content batch 1...


Saving content batch 2...


25/07/21 15:13:04 WARN TransportChannelHandler: Exception in connection from /172.18.4.62:41361
java.io.IOException: Connection timed out
	at sun.nio.ch.FileDispatcherImpl.read0(Native Method)
	at sun.nio.ch.SocketDispatcher.read(SocketDispatcher.java:39)
	at sun.nio.ch.IOUtil.readIntoNativeBuffer(IOUtil.java:223)
	at sun.nio.ch.IOUtil.read(IOUtil.java:192)
	at sun.nio.ch.SocketChannelImpl.read(SocketChannelImpl.java:379)
	at io.netty.buffer.PooledByteBuf.setBytes(PooledByteBuf.java:254)
	at io.netty.buffer.AbstractByteBuf.writeBytes(AbstractByteBuf.java:1132)
	at io.netty.channel.socket.nio.NioSocketChannel.doReadBytes(NioSocketChannel.java:357)
	at io.netty.channel.nio.AbstractNioByteChannel$NioByteUnsafe.read(AbstractNioByteChannel.java:151)
	at io.netty.channel.nio.NioEventLoop.processSelectedKey(NioEventLoop.java:788)
	at io.netty.channel.nio.NioEventLoop.processSelectedKeysOptimized(NioEventLoop.java:724)
	at io.netty.channel.nio.NioEventLoop.processSelectedKeys(NioEventLoop.java